## 1. Analyse de la structure du code

**a) Étapes principales et gestion de la mémoire**

- **Allocation et initialisation** : Le programme alloue la mémoire sur le CPU pour les matrices A, B et C, et initialise A et B avec des valeurs aléatoires.
- **Création du contexte cuBLAS** : Un handle (`cublasHandle_t`) est créé avec `cublasCreate` pour gérer l'état de la bibliothèque cuBLAS.
- **Allocation GPU et transfert** : La mémoire est allouée sur le GPU pour les copies de A, B et C à l'aide de `cudaMalloc`. Les données de A et B sont transférées du CPU vers le GPU via `cudaMemcpy`.
- **Calcul sur GPU** : La multiplication matricielle est effectuée sur le GPU en appelant `cublasSgemm`.
- **Retour des résultats** : La matrice résultat C est copiée du GPU vers le CPU.
- **Libération des ressources** : Les mémoires allouées sur le GPU et le CPU sont libérées, et le handle cuBLAS est détruit avec `cublasDestroy`.

La gestion explicite de la mémoire entre le CPU et le GPU est nécessaire car ces deux entités disposent de mémoires physiques séparées. Les données doivent être transférées explicitement pour être accessibles par le GPU.

**b) Rôle du `cublasHandle_t`**

Le `cublasHandle_t` est un objet qui représente le contexte d'exécution de la bibliothèque cuBLAS. Il permet de conserver l'état et les configurations nécessaires pour exécuter les fonctions de la bibliothèque. `cublasCreate` initialise ce contexte et alloue les ressources nécessaires, tandis que `cublasDestroy` libère ces ressources une fois les opérations terminées.

**c) Signification des paramètres `CUBLAS_OP_N`**

Les paramètres `CUBLAS_OP_N` indiquent que les matrices ne doivent pas être transposées lors de l'appel à `cublasSgemm`. Si l’un d’eux était remplacé par `CUBLAS_OP_T`, la matrice correspondante serait transposée avant la multiplication, ce qui modifierait l'ordre des éléments et pourrait changer les dimensions ou les valeurs du résultat final.

**d) Impact de l’ordre de stockage (row-major vs column-major)**

CuBLAS utilise par défaut le format column-major (comme en Fortran), alors que le C standard emploie souvent le format row-major. Ce décalage peut entraîner des interprétations erronées des matrices lors du calcul. Il faut alors soit adapter les paramètres de la fonction (par exemple, en transposant les matrices logiquement), soit convertir explicitement les données.

## 2. Comparaison cuBLAS vs calcul natif

**a) Implémentation native vs cuBLAS**

Voir parti Code

**b) Mesure du temps d’exécution et comparaison des performances**
Voir parti Code


**c) Difficultés d'une version parallèle native sur GPU**

Écrire une version native en CUDA impliquerait de gérer manuellement :
- L'organisation des threads et des blocs pour assurer une utilisation efficace des cœurs GPU.
- La gestion de la mémoire partagée pour optimiser les accès aux données.
- La synchronisation entre threads et la minimisation des accès non coalescés.

Ces aspects sont abstraits par cuBLAS, qui offre une implémentation hautement optimisée sans que l'utilisateur ait à gérer ces détails complexes.

**d) Pertinence de cuBLAS pour de petites matrices**

Pour des matrices de petite taille (ex. N = 16), les surcharges liées au lancement de kernels et aux transferts de données entre le CPU et le GPU peuvent être significatives, au point que l'avantage de l'accélération par GPU est perdu. Dans ce cas, une implémentation native sur CPU peut être plus efficace.

In [1]:
%%writefile large_matrix_mul.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

// Utility function declarations
void initializeMatrix(float *matrix, int rows, int cols);
void printMatrixSubset(float *matrix, int rows, int cols, const char *name);
void cpuMatrixMultiply(float *A, float *B, float *C, int m, int n, int k);
void compareResults(float *cpuResult, float *gpuResult, int size);

int main() {
    // Matrix dimensions
    int N = 1024;
    
    // Allocate host memory
    float *h_A = (float*)malloc(N * N * sizeof(float));
    float *h_B = (float*)malloc(N * N * sizeof(float));
    float *h_C_cpu = (float*)malloc(N * N * sizeof(float));
    float *h_C_gpu = (float*)malloc(N * N * sizeof(float));
    
    if (!h_A || !h_B || !h_C_cpu || !h_C_gpu) {
        fprintf(stderr, "Host memory allocation failed\n");
        return 1;
    }
    
    // Initialize matrices
    printf("Initializing matrices...\n");
    initializeMatrix(h_A, N, N);
    initializeMatrix(h_B, N, N);
    
    // Print small subsets to verify data
    printMatrixSubset(h_A, N, N, "Matrix A (subset)");
    printMatrixSubset(h_B, N, N, "Matrix B (subset)");
    
    // ---------------- CPU Matrix Multiplication ----------------
    printf("Performing CPU matrix multiplication...\n");
    
    clock_t cpu_start = clock();
    cpuMatrixMultiply(h_A, h_B, h_C_cpu, N, N, N);
    clock_t cpu_end = clock();
    
    double cpu_time = ((double)(cpu_end - cpu_start)) / CLOCKS_PER_SEC;
    printf("CPU matrix multiplication completed in %.3f seconds\n", cpu_time);
    
    // ---------------- cuBLAS Matrix Multiplication ----------------
    printf("Performing GPU matrix multiplication with cuBLAS...\n");
    
    // Allocate device memory
    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, N * N * sizeof(float));
    cudaMalloc((void**)&d_B, N * N * sizeof(float));
    cudaMalloc((void**)&d_C, N * N * sizeof(float));
    
    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Create cuBLAS handle
    cublasHandle_t handle;
    cublasCreate(&handle);
    
    // Copy matrices from host to device
    cudaMemcpy(d_A, h_A, N * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, N * N * sizeof(float), cudaMemcpyHostToDevice);
    
    // Perform matrix multiplication using cuBLAS
    float alpha = 1.0f;
    float beta = 0.0f;
    
    cudaEventRecord(start);
    
    // Note: cuBLAS uses column-major order, so we compute B * A instead of A * B
    // C = alpha*op(A)*op(B) + beta*C
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, 
                N, N, N, 
                &alpha, 
                d_B, N,  // Matrix B
                d_A, N,  // Matrix A
                &beta, 
                d_C, N); // Matrix C result
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    // Copy result from device to host
    cudaMemcpy(h_C_gpu, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);
    
    // Calculate elapsed time
    float gpu_time = 0;
    cudaEventElapsedTime(&gpu_time, start, stop);
    printf("GPU matrix multiplication completed in %.3f seconds\n", gpu_time/1000.0);
    
    // Compare results
    printMatrixSubset(h_C_cpu, N, N, "CPU Result (subset)");
    printMatrixSubset(h_C_gpu, N, N, "GPU Result (subset)");
    compareResults(h_C_cpu, h_C_gpu, N * N);
    
    // Print performance comparison
    printf("\nPerformance Comparison:\n");
    printf("CPU time: %.3f seconds\n", cpu_time);
    printf("GPU time: %.3f seconds\n", gpu_time/1000.0);
    printf("Speedup: %.2fx\n", cpu_time/(gpu_time/1000.0));
    
    // Calculate GFLOPS (Giga Floating Point Operations Per Second)
    double cpu_gflops = (2.0 * N * N * N) / (cpu_time * 1e9);
    double gpu_gflops = (2.0 * N * N * N) / ((gpu_time/1000.0) * 1e9);
    printf("CPU Performance: %.2f GFLOPS\n", cpu_gflops);
    printf("GPU Performance: %.2f GFLOPS\n", gpu_gflops);
    
    // Cleanup
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gpu);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cublasDestroy(handle);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    printf("\n--- Complexity Analysis ---\n");
    printf("Native CPU Implementation: O(N³) complexity\n");
    printf("- Triple-nested loops iterating through all matrix elements\n");
    printf("- No optimization for cache locality or parallelism\n\n");
    
    printf("cuBLAS Implementation: Effectively O(N³) but highly optimized\n");
    printf("- Uses tiling to maximize cache utilization\n");
    printf("- Employs thousands of parallel threads on GPU\n");
    printf("- Utilizes specialized matrix multiplication hardware (Tensor Cores if available)\n");
    printf("- Implements advanced blocking strategies to minimize memory access latency\n");
    printf("- Benefits from decades of research in optimizing matrix operations\n");
    
    return 0;
}

// Initialize matrix with random values
void initializeMatrix(float *matrix, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            matrix[i * cols + j] = (float)(rand() % 100) / 100.0f;
        }
    }
}

// Print a small subset of the matrix to verify data
void printMatrixSubset(float *matrix, int rows, int cols, const char *name) {
    printf("%s (3x3 corner):\n", name);
    int display_size = 3;
    for (int i = 0; i < display_size && i < rows; i++) {
        for (int j = 0; j < display_size && j < cols; j++) {
            printf("%.4f ", matrix[i * cols + j]);
        }
        printf("\n");
    }
    printf("\n");
}

// Native CPU matrix multiplication implementation
void cpuMatrixMultiply(float *A, float *B, float *C, int m, int n, int k) {
    // A: m x k matrix
    // B: k x n matrix
    // C: m x n matrix (result)
    
    // Classic triple loop matrix multiplication
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            float sum = 0.0f;
            for (int p = 0; p < k; p++) {
                sum += A[i * k + p] * B[p * n + j];
            }
            C[i * n + j] = sum;
        }
    }
}

// Compare CPU and GPU results for accuracy
void compareResults(float *cpuResult, float *gpuResult, int size) {
    float epsilon = 1e-3;  // Tolerance for floating point comparison
    int errors = 0;
    
    for (int i = 0; i < size; i++) {
        if (fabs(cpuResult[i] - gpuResult[i]) > epsilon) {
            errors++;
            if (errors < 10) {
                printf("Error at index %d: CPU = %f, GPU = %f\n", 
                      i, cpuResult[i], gpuResult[i]);
            }
        }
    }
    
    if (errors > 0) {
        printf("Found %d errors (tolerance: %e)\n", errors, epsilon);
    } else {
        printf("Results match! No errors found.\n");
    }
}

Writing large_matrix_mul.cu


In [ ]:
!nvcc -o matrix_multiply matrix_multiply.cu -lcublas
! ./matrix_multiply


## 3. Gestion de la mémoire

**a) Rôle de `cudaMalloc` et `cudaMemcpy`**

Les fonctions `cudaMalloc` et `cudaMemcpy` sont utilisées pour allouer et transférer explicitement les données dans l'espace mémoire du GPU, qui est distinct de celui du CPU. Si on omettait l'appel à `cudaMemcpy` avant `cublasSgemm`, le GPU ne disposerait pas des données initialisées sur le CPU, ce qui conduirait à des erreurs ou à des résultats incorrects.

**b) Calcul de la mémoire GPU allouée pour N = 1024**

Pour chaque matrice :
- Taille = 1024 x 1024 x 4 bytes (pour un float) ≈ 4 Mo.

Pour 3 matrices (A, B, C) : environ 12 Mo au total.

Si N doublait (N = 2048), la mémoire par matrice serait proportionnelle à N², donc environ 4 fois plus élevée, soit environ 48 Mo pour les trois matrices.

**c) Risques en oubliant `cudaFree`**

Omettre d'appeler `cudaFree` pour libérer `d_A`, `d_B` et `d_C` provoquerait une fuite de mémoire sur le GPU, réduisant ainsi la mémoire disponible pour d'autres calculs. On peut vérifier la libération de la mémoire en utilisant des outils comme `nvidia-smi`, qui affichent l'utilisation de la mémoire GPU.

**d) Différence entre `cudaMallocManaged` et `cudaMalloc`**

`cudaMallocManaged` alloue une mémoire unifiée accessible à la fois par le CPU et le GPU, simplifiant ainsi le transfert de données. En revanche, `cudaMalloc` réserve de la mémoire exclusivement sur le GPU, nécessitant des transferts explicites via `cudaMemcpy`.

## 4. Paramètres de cuBLAS

**a) Rôle des paramètres `alpha` et `beta`**

Dans l'appel à `cublasSgemm`, le calcul réalisé est :

    C = alpha * A * B + beta * C

Les paramètres `alpha` et `beta` permettent de scaler respectivement le produit matriciel et la matrice C déjà existante. Si `beta` était fixé à 1.0f sans initialiser C, les valeurs non définies de C seraient ajoutées au résultat, entraînant des erreurs.

**b) Modification pour calculer C = 2A * B + 3C**

Il suffit de remplacer `alpha` par 2.0f et `beta` par 3.0f dans l'appel à `cublasSgemm`.

**c) Adaptation pour des matrices non carrées**

Pour multiplier, par exemple, une matrice A de taille 512x1024 par une matrice B de taille 1024x768, il faut adapter les dimensions dans l'appel à `cublasSgemm` :
- m = 512 (nombre de lignes de A),
- n = 768 (nombre de colonnes de B),
- k = 1024 (nombre de colonnes de A ou lignes de B).

Les paramètres de stride et les leading dimensions doivent également être ajustés en conséquence.

**d) Fonctions batched dans cuBLAS**

CuBLAS propose des fonctions pour la multiplication de matrices en mode batched, telles que `cublasSgemmBatched` et `cublasSgemmStridedBatched`, permettant d'exécuter plusieurs multiplications en parallèle en passant des tableaux de pointeurs vers les matrices ou en utilisant un stride constant.

## 5. Optimisation et performance

**a) Pourquoi cuBLAS est-il plus rapide pour N = 1024 ?**

CuBLAS exploite le parallélisme massif des GPU, utilisant des milliers de cœurs et des optimisations spécifiques (comme la gestion efficace des accès mémoire et l'utilisation de Tensor Cores sur certaines architectures) pour accélérer les calculs de grande envergure.

**b) Avantage de cuBLAS pour de petites matrices (ex. N = 16)**

Pour des matrices de très petite taille, l'overhead associé aux lancements de kernels et aux transferts de données entre CPU et GPU peut surpasser les bénéfices du calcul parallèle. Ainsi, pour N = 16, une implémentation CPU native pourrait être plus performante en raison de la faible charge de calcul et de l'absence d'overhead de transfert.

**c) Traitement de matrices trop grandes pour la mémoire GPU**

Pour des matrices trop grandes (par exemple, N = 10000), il est pertinent de découper le problème en sous-blocs (technique de tiling). Chaque bloc est multiplié séparément sur le GPU et les résultats sont ensuite assemblés pour former la matrice finale. Cette approche permet de traiter des matrices dont la taille excède la mémoire disponible sur le GPU.

**d) Impact du choix des blocs et des threads dans un kernel CUDA personnalisé**

Le choix de la taille des blocs et du nombre de threads influe directement sur l'efficacité de l'utilisation des ressources GPU, la coalescence des accès mémoire et l'utilisation de la mémoire partagée. Une mauvaise configuration peut entraîner une sous-utilisation des capacités du GPU et une dégradation significative des performances.

## 6. Extensions et réflexion

**a) Passage en double précision**

Pour utiliser des nombres en double précision, il faut :
- Remplacer le type `float` par `double` dans la déclaration des matrices.
- Utiliser `cublasDgemm` au lieu de `cublasSgemm`.
- Adapter les constantes `alpha` et `beta` (par exemple, 1.0f deviendra 1.0 en double).

**b) Optimisation pour des multiplications consécutives**

Si plusieurs multiplications doivent être réalisées (par exemple, C = A * B suivi de D = C * E), il est avantageux de conserver les matrices intermédiaires dans la mémoire GPU afin de minimiser les transferts entre le CPU et le GPU.

**c) Utilisation de plusieurs GPU**

Pour exploiter plusieurs GPU, il est nécessaire de partitionner les matrices et de distribuer les calculs entre les différents dispositifs. Les principaux défis incluent la synchronisation inter-GPU, la gestion des transferts de données entre GPU, et la répartition équilibrée de la charge de travail.

**d) Optimisations sur les architectures récentes**

CuBLAS intègre des optimisations pour les architectures récentes comme Ampere ou Hopper, notamment via l'utilisation des Tensor Cores et des fonctionnalités avancées offertes par cuBLASLt, qui permettent un contrôle plus fin et des performances accrues pour certaines opérations de multiplication matricielle.